## Reconhecimento de digitais

In [116]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import pickle

In [117]:
def writeFeatures(kp, des, fileName):
    index1 = []
    for point in kp:
        temp = (point.pt, point.size, point.angle, point.response, point.octave, 
            point.class_id) 
        index1.append(temp)
    file=open("keypoints/"+fileName+".pkl", "wb")
    pickle.dump(index1, file)
    file.close()
    file=open("descritores/"+fileName+".npy", "wb")
    np.save(file, des)
    file.close()   

def readFeatures(fileName):
    file=open("descritores/"+fileName+".npy", "rb")
    des=np.load(file)
    file.close()

    index = pickle.load(open("keypoints/"+fileName+".pkl", "rb"))
    kp = []

    for point in index:
        temp = cv2.KeyPoint(x=point[0][0],y=point[0][1],size=point[1], angle=point[2], 
                                response=point[3], octave=point[4], class_id=point[5]) 
        kp.append(temp)
    return [kp, des]

In [118]:
def SIFT_Match_Image(file1, file2):
    img1=cv2.imread("imagens/"+file1+".tif", cv2.IMREAD_GRAYSCALE)
    img2=cv2.imread("imagens/"+file2+".tif", cv2.IMREAD_GRAYSCALE)

    sift=cv2.SIFT_create(nOctaveLayers=4, contrastThreshold=0.12, sigma=2.5, enable_precise_upscale=True) 
    #valores sugeridos pelo artigo, com sigma exagerado pra remover pontos de interesse muito pequenos, gerados por regiões de baixa pressão
    lista=readFeatures(file1)
    kp1, des1=lista[0], lista[1]
    kp2, des2=sift.detectAndCompute(img2, None)

    index_params = dict(algorithm = 1, trees = 5) #alg=1 significa usar indexação em kdtree
    search_params = dict(checks=50)  #quantidade de pontos checados pelo knn
    
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    
    matches = flann.knnMatch(des1,des2,k=2)
    
    # Need to draw only good matches, so create a mask
    matchesMask = [[0,0] for i in range(len(matches))]
    counter=0
    # ratio test as per Lowe's paper
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.8*n.distance:
            counter+=1
            matchesMask[i]=[1,0]
    
    draw_params = dict(matchColor = (0,255,0),
                    singlePointColor = (255,0,0),
                    matchesMask = matchesMask,
                    flags = cv2.DrawMatchesFlags_DEFAULT)
    
    img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,matches,None,**draw_params)
    
    plt.imshow(img3,),plt.show()
    
    return [counter, (len(kp1)+len(kp2))/2]

def SIFT_Match(kp1, des1, kp2, des2):
    index_params = dict(algorithm = 1, trees = 5) #alg=1 significa usar indexação em kdtree
    search_params = dict(checks=50)  #quantidade de pontos checados pelo knn
    
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    
    matches = flann.knnMatch(des1,des2,k=2)
    
    counter=0
    # ratio test as per Lowe's paper
    for (m,n) in matches:
        if m.distance < 0.8*n.distance:
            counter+=1

    return [counter, (len(kp1)+len(kp2))/2]

In [119]:
if not os.path.exists("keypoints"):
    os.makedirs("keypoints")

if not os.path.exists("descritores"):
    os.makedirs("descritores")

sift=cv2.SIFT_create(nOctaveLayers=4, contrastThreshold=0.12, sigma=2.5, enable_precise_upscale=True) 
for file in os.listdir("imagens"):
    img=cv2.imread("imagens/"+file, cv2.IMREAD_GRAYSCALE)
    kp, des=sift.detectAndCompute(img, None)
    file=file[0:len(file)-4]
    writeFeatures(kp, des, file)

In [120]:
from random import randint

files=[]

for str in os.listdir("imagens"):
    files+=[str[0:len(str)-4]]


dados=list()
for i in range(len(files)//8):
    dados+=[files[8*i:8*(i+1)]]

treino=list()
teste=list()
for digital in dados:
    indice=randint(0,7)
    teste+=[digital.pop(indice)]
    treino+=[digital]

In [ ]:
def classificador(treino, imagem):
    imagem=cv2.imread("imagens/"+imagem+".tif", cv2.IMREAD_GRAYSCALE)
    sift=cv2.SIFT_create(nOctaveLayers=4, contrastThreshold=0.12, sigma=2.5, enable_precise_upscale=True) 
    kp2, des2=sift.detectAndCompute(imagem, None)
    matches=[]
    for digital in treino:
        score=0
        for leitura in digital:
            args=readFeatures(leitura)
            res=SIFT_Match(args[0], args[1], kp2, des2)
            res=res[0]/res[1]
            if(res<0.02):
                score=0
                break
            score+=res
        score=score/len(digital)
        if(score>0.08):
            matches.append([score, leitura[0:3]])
    return matches

acertos=0
cont=0
falsoPositivo=0
for arq in teste:
    print(cont)
    cont+=1
    res=classificador(treino, arq)
    if res!=[]:
        if max(res, key=lambda x:x[0])[1]==arq[0:3]:
            acertos+=1
        else:
            falsoPositivo+=1
print(acertos)
print(falsoPositivo)

0
1
Entrou
sucesso
2
Entrou
sucesso
3
Entrou
sucesso
4
Entrou
sucesso
5
Entrou
sucesso
6
Entrou
sucesso
7
Entrou
sucesso
8
Entrou
sucesso
9
Entrou
sucesso
10
Entrou
sucesso
11
Entrou
sucesso
12
Entrou
sucesso
13
Entrou
sucesso
14
Entrou
sucesso
15
Entrou
sucesso
16
Entrou
sucesso
17
18
19
Entrou
sucesso
20
Entrou
sucesso
21
22
Entrou
sucesso
23
Entrou
sucesso
24
Entrou
sucesso
25
Entrou
sucesso
26
27
28
Entrou
sucesso
29
30
Entrou
sucesso
31
Entrou
sucesso
32
Entrou
sucesso
33
Entrou
sucesso
34
Entrou
sucesso
35
Entrou
sucesso
36
Entrou
sucesso
37
Entrou
sucesso
38
Entrou
sucesso
39
Entrou
sucesso
40
Entrou
sucesso
41
Entrou
sucesso
42
Entrou
sucesso
43
Entrou
sucesso
44
45
Entrou
sucesso
46
Entrou
sucesso
47
Entrou
sucesso
48
Entrou
sucesso
49
50
Entrou
sucesso
42
